In [1]:
import os
import re
import shlex
import numpy as np
import pandas as pd
import subprocess as sp
import understand as und
from pathlib import Path
from pdb import set_trace
from collections import defaultdict
from pathlib import Path

import sys
main_path = os.getcwd()
p_df = {}
print(main_path)

/home/huyqt7/Projects/PhD/data-collection/labeled_commits


In [2]:
REPO_LINKS = {
    "abinit": {
        "url": "https://github.com/abinit/abinit",
        "lang": "fortran"
    },
    "mdanalysis": {
        "url": "https://github.com/MDAnalysis/mdanalysis",
        "lang": "python"
    },
    "libmesh": {
        "url": "https://github.com/libMesh/libmesh",
        "lang": "C"
    },
    "lammps": {
        "url": "https://github.com/lammps/lammps",
        "lang": "C"
    }
}

In [235]:
#projects = ['pcmsolver', 'rmg-py', 'hoomd', 'amber', 'xenon', 'mdanalysis', 'libmesh', 'abinit', 'lammps']
def get_releases():
    p_df = {}
    for p in projects:
        print(p)
        project_path = "/home/huyqt7/Projects/PhD/eager/data/projects/%s/" % p
        os.chdir(project_path)
        #sp.check_output("git pull", shell=True)
        sp.check_output("git reset --hard master", shell=True)
        #sp.check_output("git pull", shell=True)
        versions = sp.check_output("git tag --sort=-taggerdate", shell=True)
        versions = str(versions)[2:-1]
        versions = versions.split('\\n')
        versions = [v for v in versions if v]
        #print(versions)
        re_commits = []
        p_df[p] = {}
        for ver in versions:
            v_report = sp.check_output(
                    "git log -1 --pretty=format:\"%%H;%%ad\" %s" % ver,
                    shell=True)
            re_commits.append(str(v_report)[2:-1].split(";"))
        #print(re_commits)
        p_df[p] = re_commits
    return p_df

In [63]:
from datetime import datetime 

def get_ground_truths():
    p_df = {}
    for p in projects:
        files = [filename for filename in os.listdir(p) if filename.endswith(".csv")]
        dfs = []
        for f in files:
            dfs.append(pd.read_csv(p + "/" + f, index_col=None))

        all_commits = pd.concat(dfs)
        
        all_commits['time'] = pd.to_datetime(all_commits['time'])
        all_commits = all_commits.drop_duplicates(subset=['hash'], keep="last")
        all_commits = all_commits.sort_values(by='time').reset_index()
        print(all_commits.shape[0])
        p_df[p] = all_commits
        all_commits.to_csv("ugly1_" + p + ".csv", index=False)
    return p_df 

In [236]:
p_df = get_releases()

mdanalysis
libmesh
lammps


In [245]:
os.chdir(main_path)
#truth_df = get_ground_truths()
truth_df = {}
for p in projects:
    truth_df[p] = pd.read_csv(p + "_concat.csv", index_col=None)
    truth_df[p]['time'] = pd.to_datetime(truth_df[p]['time'])
    print(truth_df[p].shape)

(3344, 6)
(8679, 6)
(7514, 6)


In [227]:
print(truth_df.keys())

dict_keys(['pcmsolver', 'rmg-py', 'hoomd', 'amber', 'xenon', 'mdanalysis', 'libmesh', 'abinit', 'lammps'])


In [246]:
from dateutil.parser import parse as dt_parse
os.chdir(main_path)
preprocessed_dfs = {}
#temp_p_df = {}
#for k, v in p_df.items():
#    if k != "pcmsolver":
#        temp_p_df[k] = v
for p, releases in p_df.items():
    preprocessed_dfs[p] = []
    temp_df_total = truth_df[p]
    temp_df = pd.DataFrame()
    temp_releases = [[r[0], dt_parse(r[1]).replace(tzinfo=None)] for r in releases]
    temp_releases = sorted(temp_releases, key=lambda r: r[1])
    #print([r[1] for r in temp_releases])
    #sorted_r = list(reversed(releases))
    #print([r[1] for r in sorted_r])
    print(p, len(temp_releases), temp_df_total.shape[0])
    for i in range(len(temp_releases)):
        r = temp_releases[i]
        time_r = r[1]
        #time_r = dt_parse(r[1]).replace(tzinfo=None)
        #print(truth_df[p]['time'].iloc[0], "hello", time_r)
        #print(time_r, temp_df_total['time'].iloc[0])
        if temp_df_total['time'].iloc[0] > time_r:
            continue
        else:
            if temp_df.shape[0] > 0:
                temp_df = pd.concat([temp_df, temp_df_total[temp_df_total['time'] < time_r]])
            else:
                #print(temp_df_total['time'] <= time_r)
                temp_df = temp_df_total[temp_df_total['time'] < time_r]
            temp_df_total = temp_df_total[temp_df_total['time'] >= time_r]
            if temp_df.shape[0] > (float(truth_df[p].shape[0]) * 0.1):
                preprocessed_dfs[p].append(temp_df)
                temp_df.to_csv("preprocessed_commits/%s/%s_%s.csv" % (p, p, str(len(preprocessed_dfs[p]))), index=False)
                temp_df = pd.DataFrame()
            else:
                if i == len(temp_releases) - 1:
                    preprocessed_dfs[p][-1] = pd.concat([preprocessed_dfs[p][-1], temp_df])
                    preprocessed_dfs[p][-1].to_csv("preprocessed_commits/%s/%s_%s.csv" % (p, p, str(len(preprocessed_dfs[p]))), index=False)
            #print(p, temp_df.shape, time_r, len(preprocessed_dfs[p]), preprocessed_dfs[p][-1].shape)
                elif temp_df_total.shape[0] < 1:
                    preprocessed_dfs[p][-1] = pd.concat([preprocessed_dfs[p][-1], temp_df])
                    preprocessed_dfs[p][-1].to_csv("preprocessed_commits/%s/%s_%s.csv" % (p, p, str(len(preprocessed_dfs[p]))), index=False)
                    break
    print(p, len(temp_releases), truth_df[p].shape[0], [x.shape[0] for x in preprocessed_dfs[p]])

mdanalysis 48 3344
mdanalysis 48 3344 [352, 339, 392, 355, 426, 429, 403, 648]
libmesh 59 8679
libmesh 59 8679 [1125, 1326, 1339, 947, 1062, 983, 1897]
lammps 93 7514
lammps 93 7514 [807, 883, 914, 1180, 855, 754, 837, 1284]


In [230]:
for p in ['pcmsolver', 'amber']:
    temp_dfs = []
    if p == 'pcmsolver':
        len_df = preprocessed_dfs[p][0].shape[0]
        temp_dfs = [preprocessed_dfs[p][0][:int(len_df/3)], 
                    preprocessed_dfs[p][0][int(len_df/3):int(2*len_df/3)],
                    preprocessed_dfs[p][0][int(2*len_df/3):],
                    preprocessed_dfs[p][1]]
        print(temp_dfs[0].shape[0],temp_dfs[1].shape[0], temp_dfs[2].shape[0], temp_dfs[3].shape[0])
    else:
        len_df = preprocessed_dfs[p][0].shape[0]
        temp_dfs = [preprocessed_dfs[p][0][:int(len_df/2)], 
                    preprocessed_dfs[p][0][int(len_df/2):],
                    preprocessed_dfs[p][1],preprocessed_dfs[p][2]]
        print(temp_dfs[0].shape[0],temp_dfs[1].shape[0], temp_dfs[2].shape[0], temp_dfs[3].shape[0])
    index = 1
    for df in temp_dfs:
        df.to_csv("preprocessed_commits/%s/%s_%s.csv" % (p, p, index), index=False)
        index += 1

522 523 523 272
1165 1165 1019 1538


In [122]:
datetime.fromtimestamp(int(float('1540317256.69321'))).replace(tzinfo=None)

datetime.datetime(2018, 10, 23, 13, 54, 16)

In [131]:
five_projects = ['pcmsolver', 'rmg-py', 'hoomd', 'amber', 'xenon']
os.chdir(main_path)
for p in five_projects:
    p_files = [filename for filename in os.listdir('raw') if filename.startswith(p)]
    fr_files = [filename for filename in os.listdir('fastread') if filename.startswith(p)]
    raw_df = pd.read_csv("auto/" + p_files[0], index_col=None)
    #raw_df['Year'] = pd.to_datetime(raw_df['Year'])
    #raw_df = raw_df[-5000:]
    #raw_df.to_csv("preprocessed/raw_" + p + ".csv", index=False)
    fr_df = pd.read_csv("fastread/" + fr_files[0], index_col=None)
    print(raw_df.shape[0], fr_df.shape[0])

328 1844
5000 5000
5000 5000
5000 5000
2315 2315


In [140]:
five_pdf = {}
projects = ['mdanalysis', 'libmesh', 'lammps']

for p in projects:
    five_pdf[p] = []
    timestamps = []
    hashvalues = []
    col_names = ['hash', 'time', 'message', 'keyword_buggy', 'human_buggy', 'fastread_buggy']
    p_df_final = pd.DataFrame(columns=col_names)
    p_df_concat = pd.read_csv("fastread/" + p + "_fast_labeled.csv")
    p_df_concat['fr_labels'] = pd.Series(np.where(p_df_concat.code == 'yes', 1, 0), p_df_concat.index)
    p_df_fast = pd.read_csv("raw/" + p + "_fast_labeled.csv")
    p_df_fast['Year'] = pd.to_datetime(p_df_fast['Year'])
    for h in p_df_concat['Document Title'].values.tolist():
        if len(h) > 7:
            h = h[:7]
        val = p_df_fast[p_df_fast['Document Title'].str.startswith(h)]
        if not val.empty:
            for i, row in val.iterrows():
                if row['Document Title'][-2] != ".0":            
                    timestamps.append(val['Year'].values.tolist()[0])
                    hashvalues.append(val['Document Title'].values.tolist()[0])
        else: 
            print(h)
    p_df_final['hash'] = hashvalues
    p_df_final['time'] = timestamps
    p_df_final['time'] = pd.to_datetime(p_df_final['time'])
    p_df_final['message'] = p_df_concat['Abstract']
    p_df_final['fastread_buggy'] = p_df_concat['fr_labels']
    p_df_final = p_df_final.sort_values(by='time').reset_index()
    p_df_final.to_csv(p + "_concat.csv", index=False)
    print(p, p_df_final.shape[0])
    print(p_df_final.groupby('fastread_buggy').count())

pcmsolver 1844
                index  hash  time  message  keyword_buggy  human_buggy
fastread_buggy                                                        
0                1650  1650  1650     1650              0            0
1                 194   194   194      194              0            0
rmg-py 5000
                index  hash  time  message  keyword_buggy  human_buggy
fastread_buggy                                                        
0                4347  4347  4347     4347              0            0
1                 653   653   653      653              0            0
hoomd 5000
                index  hash  time  message  keyword_buggy  human_buggy
fastread_buggy                                                        
0                4236  4236  4236     4236              0            0
1                 764   764   764      764              0            0
amber 5000
                index  hash  time  message  keyword_buggy  human_buggy
fastread_buggy              

In [143]:
projects = ['pcmsolver', 'rmg-py', 'hoomd', 'amber', 'xenon', 'mdanalysis', 'libmesh', 'abinit', 'lammps']

In [242]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import string

ps = PorterStemmer()
def keyword_buggy():
    for p in projects:
        index = 0
        all_commits = pd.read_csv(p + "_concat.csv", index_col=None)
        print(all_commits.shape)
        commits_messages = all_commits['message'].values.tolist()
        buggy = [0]*all_commits.shape[0]
        for i in range(1, all_commits.shape[0]):
            m = commits_messages[i]
            try:
                words = word_tokenize(m)
                stemmed_w = [ps.stem(w) for w in words]
                bug_key_words = set(["bug", "fix", "wrong", "error", "fail", "problem", "patch"])
                check = set(stemmed_w).intersection(bug_key_words)
                if len(check) > 0:
                    buggy[i] = 1
            except TypeError:
                index += 1
                #print("TypeError", i, p, m)
                continue
            except UnicodeDecodeError:
                index += 1
                #print("UDError", i, p, words)
                continue
        print(index)
            #print(stemmed_w)
            
                    #     Buggy commit hash              Clean commit hash
                    #(all_commits.iloc[i-1]['hash'], all_commits.iloc[i]['hash']))
        all_commits['keyword_buggy'] = buggy
        all_commits.to_csv(p + "_concat.csv", index=False)

In [244]:
for p in projects:
    all_commits = pd.read_csv(p + "_concat.csv", index_col=None)
    while(all_commits.shape[1] > 6):
        all_commits = all_commits.drop(all_commits.columns[0], axis=1)
    print(all_commits.columns)
    all_commits.to_csv(p + "_concat.csv", index=False)

Index(['hash', 'time', 'message', 'keyword_buggy', 'human_buggy',
       'fastread_buggy'],
      dtype='object')
Index(['hash', 'time', 'message', 'keyword_buggy', 'human_buggy',
       'fastread_buggy'],
      dtype='object')
Index(['hash', 'time', 'message', 'keyword_buggy', 'human_buggy',
       'fastread_buggy'],
      dtype='object')


In [218]:
sum([2166, 652, 568, 1430])

4816